In [ ]:
!pip install spacy-udpipe

In [ ]:
!pip install stanza

In [ ]:
!pip install natasha

In [15]:
import csv

In [22]:
import spacy_udpipe

In [41]:
import stanza

In [47]:
from natasha import (Segmenter, NewsEmbedding, NewsMorphTagger, Doc)

In [10]:
f = open("mis.txt", "r")
raw = f.read()
f.close()

In [11]:
raw

'Как видите, все арго — и то, которое было в ходу четыреста лет тому назад, и современное — насквозь проникнуты тем символическим духом, который придает словам то вид угрозы, то вид страдания и печали. В нем так и чувствуется дикая печаль тех жалких бродяг Двора чудес прежних времен, которые играли в карты особого образца. Некоторые из таких карт дошли до нас. Например, трефовая восьмерка изображалась на них в виде большого дерева с огромными трилистниками — символ леса. Под этим деревом виден костер, на котором три зайца поджаривают насаженного на вертел охотника, а на другом костре стоит дымящийся котелок, из которого выглядывает голова собаки. Что может быть зловещее этих карточных изображений с кострами для поджаривания контрабандистов и кипящими котлами для фальшивомонетчиков? Различные формы, в которые облекалась мысль в царстве арго, сама песня, сама насмешка, даже сама угроза, — все носит на себе характер бессилия и подавленности. Все песни арго, напевы которых сохранились до н

Токенизация (для разметки вручную) и чтение файла с готовой разметкой.

In [12]:
raw_splt = raw.split(' ')
tokens = []
for x in raw_splt:
  x = x.strip('''"   ?!/.,.—-()\n''').lower()
  if x != '':
    tokens.append(x)

In [ ]:
tokens

In [16]:
f = open('0.csv', 'w')
writer = csv.writer(f)
for x in tokens:
  writer.writerow([x])
f.close()

In [33]:
pos0 = list()
f = open('0_pos.csv', 'r') 
reader = csv.reader(f, delimiter=',')
for x in reader:
  pos0.append(x)
f.close()

Список с результатами моей разметки.

In [34]:
pos0

[['как', 'SCONJ'],
 ['видите', 'VERB'],
 ['все', 'DET'],
 ['арго', 'NOUN'],
 ['и', 'CCONJ'],
 ['то', 'PRON'],
 ['которое', 'PRON'],
 ['было', 'VERB'],
 ['в', 'ADP'],
 ['ходу', 'NOUN'],
 ['четыреста', 'NUM'],
 ['лет', 'NOUN'],
 ['тому', 'PRON'],
 ['назад', 'ADV'],
 ['и', 'CCONJ'],
 ['современное', 'ADJ'],
 ['насквозь', 'ADV'],
 ['проникнуты', 'VERB'],
 ['тем', 'DET'],
 ['символическим', 'ADJ'],
 ['духом', 'NOUN'],
 ['который', 'PRON'],
 ['придает', 'VERB'],
 ['словам', 'NOUN'],
 ['то', 'CCONJ'],
 ['вид', 'NOUN'],
 ['угрозы', 'NOUN'],
 ['то', 'CCONJ'],
 ['вид', 'NOUN'],
 ['страдания', 'NOUN'],
 ['и', 'CCONJ'],
 ['печали', 'NOUN'],
 ['в', 'ADP'],
 ['нем', 'PRON'],
 ['так', 'ADV'],
 ['и', 'PART'],
 ['чувствуется', 'VERB'],
 ['дикая', 'ADJ'],
 ['печаль', 'NOUN'],
 ['тех', 'DET'],
 ['жалких', 'ADJ'],
 ['бродяг', 'NOUN'],
 ['двора', 'NOUN'],
 ['чудес', 'NOUN'],
 ['прежних', 'ADJ'],
 ['времен', 'NOUN'],
 ['которые', 'PRON'],
 ['играли', 'VERB'],
 ['в', 'ADP'],
 ['карты', 'NOUN'],
 ['особого', 

mistakes - словарь со списками ошибок каждого разметчика.

In [35]:
mistakes = dict()

Разметка с помощью UDPIPE, запись результатов в файл 1_pos.csv.

In [ ]:
spacy_udpipe.download(lang="ru")
nlp1 = spacy_udpipe.load(lang="ru")
doc = nlp1(raw)

In [38]:
f = open('1_pos.csv', 'w')
writer = csv.writer(f)
count = 0
mistakes['1'] = list()
for token in doc:
  if token.pos_ != 'PUNCT':
    writer.writerow([token.text.lower(), token.pos_])
    if token.pos_ != pos0[count][1]:
      mistakes['1'].append([count, token.pos_])
    count += 1
f.close()

Разметка с помощью STANZA, запись результатов в файл 2_pos.csv.

In [ ]:
nlp2 = stanza.Pipeline(lang='ru', processors='tokenize,pos')
doc = nlp2(raw)

In [44]:
f = open('2_pos.csv', 'w')
writer = csv.writer(f)
count = 0
mistakes['2'] = list()
for sent in doc.sentences:
  for token in sent.words:
    if token.upos != 'PUNCT':
      writer.writerow([token.text.lower(), token.upos])
      if token.upos != pos0[count][1]:
        mistakes['2'].append([count, token.upos])
      count += 1
f.close()

Разметка с помощью NATASHA, запись результатов в файл 1_pos.csv.

In [48]:
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
doc = Doc(raw)
segmenter = Segmenter()
doc.segment(segmenter)
morph_tagger = NewsMorphTagger(emb)
doc.tag_morph(morph_tagger)

In [49]:
f = open('3_pos.csv', 'w')
writer = csv.writer(f)
count = 0
mistakes['3'] = list()
for token in doc.tokens:
  if token.pos != 'PUNCT':
    writer.writerow([token.text.lower(), token.pos])
    if token.pos != pos0[count][1]:
      mistakes['3'].append([count, token.pos])
    count += 1
f.close()


Функция для подсчёта accuracy и вывода ошибок.

In [55]:
def accuracy(num):
  all = len(pos0)
  for token_m in mistakes[str(num)]:
    print('токен: ', pos0[token_m[0]][0], '\nправильный ответ: ', pos0[token_m[0]][1], '\nответ модели: ', token_m[1], '\n')
  return 1 - len(mistakes[str(num)]) / all 

UDPIPE

In [57]:
print('UDPIPE accuracy score: ', accuracy(1))

токен:  было 
правильный ответ:  VERB 
ответ модели:  AUX 

токен:  то 
правильный ответ:  CCONJ 
ответ модели:  SCONJ 

токен:  то 
правильный ответ:  CCONJ 
ответ модели:  SCONJ 

токен:  жалких 
правильный ответ:  ADJ 
ответ модели:  DET 

токен:  двора 
правильный ответ:  NOUN 
ответ модели:  PROPN 

токен:  быть 
правильный ответ:  VERB 
ответ модели:  AUX 

токен:  зловещее 
правильный ответ:  ADJ 
ответ модели:  NOUN 

токен:  сама 
правильный ответ:  DET 
ответ модели:  ADJ 

токен:  сама 
правильный ответ:  PRON 
ответ модели:  ADJ 

токен:  насмешка 
правильный ответ:  NOUN 
ответ модели:  ADV 

токен:  сама 
правильный ответ:  DET 
ответ модели:  ADJ 

токен:  были 
правильный ответ:  VERB 
ответ модели:  AUX 

токен:  обездоленный 
правильный ответ:  NOUN 
ответ модели:  VERB 

токен:  едва 
правильный ответ:  PART 
ответ модели:  ADV 

токен:  и 
правильный ответ:  CCONJ 
ответ модели:  PART 

токен:  как 
правильный ответ:  ADV 
ответ модели:  SCONJ 

токен:  как 
правиль

STANZA

In [58]:
print('STANZA accuracy score: ', accuracy(2))

токен:  было 
правильный ответ:  VERB 
ответ модели:  AUX 

токен:  то 
правильный ответ:  CCONJ 
ответ модели:  SCONJ 

токен:  то 
правильный ответ:  CCONJ 
ответ модели:  SCONJ 

токен:  двора 
правильный ответ:  NOUN 
ответ модели:  PROPN 

токен:  зловещее 
правильный ответ:  ADJ 
ответ модели:  VERB 

токен:  кипящими 
правильный ответ:  VERB 
ответ модели:  ADJ 

токен:  сама 
правильный ответ:  DET 
ответ модели:  ADJ 

токен:  сама 
правильный ответ:  PRON 
ответ модели:  ADJ 

токен:  сама 
правильный ответ:  DET 
ответ модели:  ADJ 

токен:  были 
правильный ответ:  VERB 
ответ модели:  AUX 

токен:  обездоленный 
правильный ответ:  NOUN 
ответ модели:  VERB 

токен:  одними 
правильный ответ:  DET 
ответ модели:  NUM 

токен:  как 
правильный ответ:  ADV 
ответ модели:  SCONJ 

токен:  самому 
правильный ответ:  PRON 
ответ модели:  ADJ 

токен:  жалобное 
правильный ответ:  NOUN 
ответ модели:  ADJ 

токен:  разухабистым 
правильный ответ:  NOUN 
ответ модели:  ADJ 

токен

NATASHA

In [59]:
print('NATASHA accuracy score: ', accuracy(3))

токен:  как 
правильный ответ:  SCONJ 
ответ модели:  ADV 

токен:  было 
правильный ответ:  VERB 
ответ модели:  AUX 

токен:  то 
правильный ответ:  CCONJ 
ответ модели:  SCONJ 

токен:  то 
правильный ответ:  CCONJ 
ответ модели:  SCONJ 

токен:  двора 
правильный ответ:  NOUN 
ответ модели:  PROPN 

токен:  поджаривают 
правильный ответ:  VERB 
ответ модели:  ADJ 

токен:  насаженного 
правильный ответ:  VERB 
ответ модели:  NOUN 

токен:  дымящийся 
правильный ответ:  VERB 
ответ модели:  ADJ 

токен:  зловещее 
правильный ответ:  ADJ 
ответ модели:  NOUN 

токен:  поджаривания 
правильный ответ:  NOUN 
ответ модели:  ADJ 

токен:  кипящими 
правильный ответ:  VERB 
ответ модели:  ADJ 

токен:  сама 
правильный ответ:  DET 
ответ модели:  ADJ 

токен:  сама 
правильный ответ:  PRON 
ответ модели:  ADJ 

токен:  сама 
правильный ответ:  DET 
ответ модели:  ADJ 

токен:  были 
правильный ответ:  VERB 
ответ модели:  AUX 

токен:  обездоленный 
правильный ответ:  NOUN 
ответ модели: 